In [2]:
import brightway2 as bw
from brightway2 import *

In [3]:
# to import forwast
import os
import zipfile
from bw2data.utils import download_file

In [4]:
from premise import *

ModuleNotFoundError: No module named 'premise'

## This notebook

The objective of this notebook is to demonstrate how to solve the "My First LCA" exercise from the ESA course using the Brightway and ActivityBrowser tools. Additionally, it provides supplementary information on these tools.

- Author (and contact):
    - Alvaro Hahn (alvaro.hahn-menacho@psi.ch)


### Useful material

1. General info on how to use Brightway:
   - https://github.com/maximikos/Brightway2_Intro/blob/master/BW2_tutorial.ipynb
   - https://docs.brightway.dev/en/legacy/
3. General info on Activity-Browser:
   - https://github.com/LCA-ActivityBrowser/activity-browser
   - https://www.youtube.com/@activity-browser
4. Interested in prospective LCA 🧐:
   - https://github.com/polca/premise

### 1. Set project and create biosphere

In [5]:
bw.projects.set_current("myfirstLCA") #Creating/accessing the project
bw.bw2setup() #Importing elementary flows, LCIA methods and some other data

Writing activities to SQLite3 database:


Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/16/2024 09:14:16
  Finished: 04/16/2024 09:14:16
  Total time elapsed: 00:00:00
  CPU %: 99.70
  Memory %: 1.30
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations



## 2. Import forwast database (and ecoinvent)

In [15]:
#FORWAST

if 'forwast' in bw.databases:
    print("Database has already been imported.")
else:
    filepath = download_file("forwast.bw2package.zip", url="http://lca-net.com/wp-content/uploads/")
    dirpath = os.path.dirname(filepath)
    zipfile.ZipFile(filepath).extractall(dirpath)
    BW2Package.import_file(os.path.join(dirpath, "forwast.bw2package"))

Database has already been imported.


In [6]:
#ecoinvent
if 'ecoinvent 3.9.1 cutoff' in bw.databases:
    print("Database has already been imported.")
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpei39cut = r"C:\Users\hahnme_a\Ecoinvent\3.9\datasets"
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    ei39cut = bw.SingleOutputEcospold2Importer(fpei39cut, 'ecoinvent 3.9.1 cutoff')
    ei39cut
    ei39cut.apply_strategies()
    ei39cut.statistics()
    ei39cut.write_database()

Extracting ecospold2 files:
0% [##############################] 100% | ETA: 00:00:00 | Item ID: fffede58-fed2-5
Total time elapsed: 00:03:13


Title: Extracting ecospold2 files:
  Started: 04/16/2024 09:15:32
  Finished: 04/16/2024 09:18:45
  Total time elapsed: 00:03:13
  CPU %: 89.30
  Memory %: 18.13
Extracted 21238 datasets in 195.12 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_un

Writing activities to SQLite3 database:


21238 datasets
674593 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Title: Writing activities to SQLite3 database:
  Started: 04/16/2024 09:18:52
  Finished: 04/16/2024 09:19:29
  Total time elapsed: 00:00:36
  CPU %: 98.50
  Memory %: 18.92
Created database: ecoinvent 3.9.1 cutoff


In [7]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.9.1 cutoff

## 3. Import Excel database

In [8]:
imp = bw.ExcelImporter(os.path.join("data/LCI_cars.xlsx"))
imp.apply_strategies()
imp.match_database(fields=('name', 'unit', 'location'))
imp.match_database("ecoinvent 3.9.1 cutoff", fields=('name', 'unit', 'location'))
imp.statistics()

Extracted 1 worksheets in 0.04 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.12 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
8 datasets
27 exchanges
0 unlinked exchanges
  


(8, 27, 0)

### ⚠️ Warning: It is key to have 0 unlinked exchanges. Otherwise, it will not be possible to solve the system.

#### Only once we have 0 unliked exchanges we write our database. What if we need to troubleshoot? We run:

In [ ]:
# imp.write_excel()

In [9]:
imp.write_database()

Writing activities to SQLite3 database:
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/16/2024 09:37:09
  Finished: 04/16/2024 09:37:09
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 19.30
Created database: esa_exercise


In [10]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.9.1 cutoff
	esa_exercise

## 4. Exploring the database

In [23]:
wbi = bw.Database("pLCA_test")
for act in wbi:
    print(act)

'driving the ICEV' (kilometer, CH, None)
'EV production' (unit, CH, None)
'battery manufacturing' (unit, CH, None)
'driving the EV' (kilometer, CH, None)
'ICEV production' (unit, CH, None)


In [24]:
for key in bw.methods:
   if 'climate change' in key:
       print(key)

('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
('Ecological Scarcity 2021', 'climate change', 'global warming potential (GWP100)')
('EF v3.0', 'climate change', 'global warming potential (GWP100)')
('EF v3.1', 'climate change', 'global warming potential (GWP100)')
('IMPACT 2002+ (Endpoint)', 'climate change', 'climate change')
('IMPACT 2002+ (Endpoint)', 'climate change', 'total')
('IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')
('IPCC 2013', 'climate change', 'global temperature change potential (GTP20)')
('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
('IPCC 2013', 'climate change', 'global warming potential (GWP20)')
('IPCC 2021', 'climate change', 'global temperature change potential (GTP100)')
('IPCC 2021', 'climate change', 'global temperature change potential (GTP50)')
('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
('IPCC 2021', 'climate change', 'global warming potential (GWP20)'

## 5. LCA

#### Goal and scope

In [25]:
# functional unit (driving 100km with each vehicle alternative)
ev_driving = [act for act in wbi if 'driving the EV' in act['name'] and 'CH' in act['location']][0]
functional_unit_EV = {ev_driving:100}
functional_unit_EV

{'driving the EV' (kilometer, CH, None): 100}

In [26]:
icev_driving = [act for act in wbi if 'driving the ICEV' in act['name'] and 'CH' in act['location']][0]
functional_unit_icev = {icev_driving:100}
functional_unit_icev

{'driving the ICEV' (kilometer, CH, None): 100}

In [27]:
# Method: let's select EF3.0
CC_method = [m for m in bw.methods if 'EF v3.0' in str(m) and  'climate change' in str(m) and 'global warming potential (GWP100)' in str(m)][0]
CC_method

('EF v3.0 no LT',
 'climate change no LT',
 'global warming potential (GWP100) no LT')

#### LCI + LCIA

In [28]:
lca_ev = LCA(functional_unit_EV,CC_method)
lca_ev.lci()
lca_ev.lcia()
lca_ev.score

0.7371395089028818

In [29]:
lca_icev = LCA(functional_unit_icev,CC_method)
lca_icev.lci()
lca_icev.lcia()
lca_icev.score

32.72776501487524

#### Interpretation

In [30]:
print('- Driving 100km using an electric car yields an impact of : {:.2f} kgCO2eq'.format(lca_ev.score))
print('- Driving 100km using a diesel car yields an impact of : {:.2f} kgCO2eq'.format(lca_icev.score))
if lca_ev.score < lca_icev.score:
    print('- Choosing an electric vehicle is the better alternative for climate change mitigation, resulting in a {:.2f}% reduction in impact compared to an internal combustion engine vehicle.'
          .format((lca_icev.score - lca_ev.score) / lca_icev.score * 100))
else:
    print('- Choosing an internal combustion engine vehicle is the better alternative for climate change mitigation, resulting in a {:.2f}% reduction in impact compared to an electric vehicle.'
          .format((lca_ev.score - lca_icev.score) / lca_ev.score * 100))

- Driving 100km using an electric car yields an impact of : 0.74 kgCO2eq
- Driving 100km using a diesel car yields an impact of : 32.73 kgCO2eq
- Choosing an electric vehicle is the better alternative for climate change mitigation, resulting in a 97.75% reduction in impact compared to an internal combustion engine vehicle.


## 6. Prospective LCA

Here comes premise

Upon the database extraction, you can import some of your Brightway2-compatible inventories like so:

In [5]:
clear_cache()

Cache folder cleared!


In [6]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2050},
                {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
            ],
            source_db="ecoinvent 3.9.1 cutoff", 
            source_version="3.9.1",
            key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
            additional_inventories= [ # <-- this is NEW
                {"filepath": r"C:\Users\hahnme_a\Desktop\ETH\Teaching LCA\Anna\pLCA_test.xlsx", "ecoinvent version": "3.9.1"}, # <-- this is NEW
            ] # <-- this is NEW
                 )

premise v.(1, 8, 0)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21238/21238 [00:00<00:00, 67496.18it/s]


Adding exchange data to activities


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 674593/674593 [00:58<00:00, 11562.36it/s]


Filling out exchange data


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21238/21238 [00:05<00:00, 3890.56it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Correct missing flow categories for biosphere exchanges
Remove empty exchanges.
Remove uncertainty data.
Done!

////////////////// IMPORTING DEFAULT INVENTORIES ///////////////////
Cannot find cached inventories. Will create them now for next time...
Importing default inventories...

Extracted 1 worksheets in 0.28 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove uncertainty data.
Extracted 1 worksheets in 0.05 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datase

C:\Users\hahnme_a\AppData\Local\miniconda3\envs\ESA_course\lib\site-packages\bw2io\importers\base_lci.py:427: UserWarning: Skipping migration migration_391_38 because it isn't installed.
  warnings.warn(
C:\Users\hahnme_a\AppData\Local\miniconda3\envs\ESA_course\lib\site-packages\bw2io\importers\base_lci.py:427: UserWarning: Skipping migration migration_391_39 because it isn't installed.
  warnings.warn(


Done!

/////////////////////// EXTRACTING IAM DATA ////////////////////////
Done!


In [7]:
ndb.update_all()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.
Done!



In [8]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
One or multiple duplicates detected. Removing them...
Database ecoinvent_cutoff_3.9_remind_SSP2-Base_2050 already exists: it will be overwritten.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:13


Title: Writing activities to SQLite3 database:
  Started: 10/24/2023 14:30:21
  Finished: 10/24/2023 14:31:35
  Total time elapsed: 00:01:13
  CPU %: 99.10
  Memory %: 31.42
Created database: ecoinvent_cutoff_3.9_remind_SSP2-Base_2050
Database ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2050 already exists: it will be overwritten.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:57


Title: Writing activities to SQLite3 database:
  Started: 10/24/2023 14:34:15
  Finished: 10/24/2023 14:35:13
  Total time elapsed: 00:00:57
  CPU %: 99.40
  Memory %: 30.21
Created database: ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2050
Generate scenario report.
Report saved under C:\Users\hahnme_a\Desktop\ETH\Teaching LCA\Anna\export\scenario_report.
Generate change report.
Report saved under C:\Users\hahnme_a\Desktop\ETH\Teaching LCA\Anna.


## LCIA